# Theoretical Recurrence Interval

__Description__:

__Input__: 

__Output__: 

---
## Load Libraries, Parameters, and File Paths:
### Libraries:

In [1]:
import os
import json
import numpy as np
import pandas as pd
import pathlib as pl
from matplotlib import pyplot as plt

### Parameters:
#### Site specific:

In [2]:
Project_Area = 'Sacramento'

#root_dir = pl.Path(os.getcwd())
root_dir = pl.Path(r'C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Efficiency70')
outputs_dir = root_dir/'Outputs'

#### Project specific (global):

In [3]:
binwidth = 0.1       # [inches]; increment for binning the excess rainfall amount
RI_max = 3000

verbose = True        # Option to display print statements
display_plots = True  # Option to display plots

### File Paths:
#### Weights:

In [4]:
weight_files = []
for f in outputs_dir.glob('*.json'):
    if 'Weights' in f.stem and 'TRI' not in f.stem: 
        weight_files.append(f)       
        print(f)

C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Efficiency70\Outputs\Sacramento_P01_D30_Weights.json
C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Efficiency70\Outputs\Sacramento_P02_D15_Weights.json
C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Efficiency70\Outputs\Sacramento_P03_D37_Weights.json


#### Forcing:

In [5]:
forcing_files = []
for f in outputs_dir.glob('**/*.json'):
    if 'Forcing' in f.parent.stem:
        forcing_files.append(f)
        print(f)

C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Efficiency70\Outputs\Sacramento_P01_Forcing\Sacramento_P01_D23.json
C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Efficiency70\Outputs\Sacramento_P01_Forcing\Sacramento_P01_D24.json
C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Efficiency70\Outputs\Sacramento_P01_Forcing\Sacramento_P01_D25.json
C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Efficiency70\Outputs\Sacramento_P01_Forcing\Sacramento_P01_D26.json
C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Efficiency70\Outputs\Sacramento_P01_Forcing\Sacramento_P01_D27.json
C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Efficiency70\Outputs\Sacramento_P01_Forcing\Sacramento_P01_D28.json
C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Efficiency70\Outputs\Sacramento_P01_Forcing\Sacramento_P01_D29.json
C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Sacramento\Efficiency70\Outputs\Sacramento_P01_Forcing\

---
## Calculate the Theoretical Recurrence Interval:

In [12]:
updated_list = []
lst = ['D30', 'D15', 'D37']
for forcing_file in forcing_files:
    for d in lst:
        if d in forcing_file.stem:
            updated_list.append(forcing_file)
updated_list            

[WindowsPath('C:/Users/sputnam/Desktop/PFRA_Production/Pluvial/Sacramento/Efficiency70/Outputs/Sacramento_P01_Forcing/Sacramento_P01_D30.json'),
 WindowsPath('C:/Users/sputnam/Desktop/PFRA_Production/Pluvial/Sacramento/Efficiency70/Outputs/Sacramento_P02_Forcing/Sacramento_P02_D15.json'),
 WindowsPath('C:/Users/sputnam/Desktop/PFRA_Production/Pluvial/Sacramento/Efficiency70/Outputs/Sacramento_P03_Forcing/Sacramento_P03_D37.json')]

In [37]:
dic = {}
dic_vol = {}
mods = []
doms = []
tris = []
small = []
first = []
large =  []
for forcing_file in updated_list:
    
    _, Pluvial_Model, Domain = forcing_file.stem.split('_')
    mods.append(Pluvial_Model)
    doms.append(Domain)
    
    for w in weight_files:
        if Pluvial_Model in w.stem:
            weight_file = w
            
    with open(forcing_file) as f:
            ff_dic =  json.load(f)
    
    with open(weight_file) as f:
            wt =  json.load(f)  
            
    mainBCN = list(wt['BCName'].keys())[0]
    
    df = pd.DataFrame()
    df['Events'] = list(wt['BCName'][mainBCN].keys())
    df['Weight'] = list(wt['BCName'][mainBCN].values())    
    df = df.set_index('Events')
    for d in list(ff_dic.keys()):
        for k, v in ff_dic[d]['BCName'][Domain].items():
            df.loc[k, 'Volume'] = sum(v)
            df.loc[k, 'Dur'] = d
    df = df.reset_index().sort_values(by=['Volume'], ascending = True).set_index('Events') 
    
    small.append(df[df['Volume']==min(df['Volume'])].index[0])
    first.append(df[df['Volume']==min(df[df['Volume']>0]['Volume'])].index[0])
    large.append(df[df['Volume']==max(df['Volume'])].index[0])
    
    binmin = min(df['Volume'])
    binmax = max(df['Volume'])
    bins = np.arange(binmin, binmax+binwidth, binwidth)
    
    lst = []
    for i, b in enumerate(bins[:]):
        if i==0:
            weight = sum(df[(0.0 == df['Volume'])]['Weight'])
        else:
            weight = sum(df[(bins[i-1]  < df['Volume']) & (df['Volume']<=b)]['Weight'])
        lst.append((b, weight))
    
    binned  = pd.DataFrame()
    binned['Volume'] = [i[0] for i in lst]
    binned['Weight'] = [i[1] for i in lst]
    binned['Cumulative'] = binned['Weight'].cumsum()
    binned['Theoretical RI'] = 1.0/(0.5-binned['Cumulative'])
    
    TTRI = binned.loc[0]['Theoretical RI']
    tris.append(TTRI)
    
    if verbose:
        print('{0} {1} Theoretical Threshold RI: {2} years'.format(Pluvial_Model, Domain, TTRI))
        
    vavg = []
    for i, b in enumerate(binned['Volume']):
        if i==0:
            vavg.append(b)
        elif i>0:
            vavg.append((b+binned['Volume'][i-1])/2)
    binned['Vavg'] = vavg 
    
    event = []
    for vavg in binned['Vavg']:
        diff = df['Volume']-vavg
        event.append(diff[diff.abs().argsort()].index[0])
    binned['Event'] = event    
    binned.head()
    
    unique_events = list(set(list(binned['Event'])))
    edf = pd.DataFrame(data = {'Event': unique_events})
    for i, e in enumerate(edf['Event']):
        edf.loc[i, 'Weight'] = sum(binned[binned['Event']==e]['Weight']) 
        edf.loc[i, 'Volume'] = df.loc[e]['Volume']
    edf = edf[edf['Weight']>0.0].copy(deep=True)
    assert np.round(sum(edf['Weight']), 2) == 0.5 , 'Total weight does not equal 0.5 as expected'
    edf = edf.sort_values('Volume').reset_index(drop = True)
    edf['Cumulative'] = edf['Weight'].cumsum()
    print('Simulations to include in the Global Scale Test: {}'.format(edf.shape[0]))
    
    #fig, ax = plt.subplots(2, 1, figsize=(18, 12))
    #ax[0].plot(df['Weight'], df['Volume'], linestyle = '', marker = '.' , label = 'All Events', color = 'gray', alpha = 0.75)
    #ax[0].plot(edf['Weight'], edf['Volume'], linestyle = '', marker = '.', color = 'blue', label = 'Subset')
    #ax[0].set_xlabel('Weight, [-]', fontsize = 10)
    #ax[0].set_ylabel('Excess Rainfall, [inches]', fontsize=10)
    #ax[0].set_xlim([0, 0.002])
    #ax[0].legend()
    #ax[0].grid()    
    #ax[1].plot(binned['Volume'], 0.5 - binned['Cumulative'], linestyle = '-', marker = '', label = 'All Events', color = 'gray', alpha = 0.75)
    #ax[1].plot(edf['Volume'], 0.5-edf['Cumulative'], linestyle = '-', marker = '', color = 'blue', label = 'Subset')
    #ax[1].set_xlabel('Excess Rainfall, [inches]', fontsize = 10)
    #ax[1].set_ylabel('Weight (~ probability)', fontsize=10)
    #ax[1].legend()
    #ax[1].grid()       
    
    edic = {'BCName':{mainBCN:{}}}
    elist =  []
    for i, e in enumerate(edf['Event']):
        edic['BCName'][mainBCN][e] = edf.loc[i]['Weight']
        elist.append(e)
    #with open(outputs_dir/'{0}_{1}_{2}_Weights_TRI.json'.format(Project_Area, Pluvial_Model, Domain), 'w') as f:
    #    json.dump(edic, f)
        
    df = pd.DataFrame(data = {'Events': elist})
    df = df.set_index('Events')
    df.to_csv(outputs_dir/'{0}_{1}_{2}_Events_TRI.csv'.format(Project_Area, Pluvial_Model, Domain))    
#tri_df = pd.DataFrame(data = {'Model': mods, 'Domain': doms, 'TRI': tris})    
#tri_df = tri_df.set_index('Model')
#tri_df.to_csv(outputs_dir/'{0}_TRI.csv'.format(Project_Area))  
scale1_df = pd.DataFrame(data = {'Model': mods, 'Domain': doms, 'Smallest': small, 'First': first, 'Largest': large})    
scale1_df = scale1_df.set_index('Model')
scale1_df.to_csv(outputs_dir/'{0}_scale1.csv'.format(Project_Area))  

P01 D30 Theoretical Threshold RI: 27.40087824732952 years
Simulations to include in the Global Scale Test: 49
P02 D15 Theoretical Threshold RI: 47.423768742270745 years
Simulations to include in the Global Scale Test: 33
P03 D37 Theoretical Threshold RI: 24.673799958892097 years
Simulations to include in the Global Scale Test: 45


---
## End